In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = [None] * 2  # Using 0-based indexing
node[0] = Delay(model, 'InfiniteServer')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[1].set_number_of_servers(2)

jobclass = [None] * 2  # Using 0-based indexing
jobclass[0] = ClosedClass(model, 'Class1', 3, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 1, node[0], 0)

node[0].set_service(jobclass[0], Exp(3))
node[0].set_service(jobclass[1], Exp(0.5))

node[1].set_service(jobclass[0], Exp(0.1))
node[1].set_service(jobclass[1], Exp(1))

M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()
# Class 1 to Class 1 routing matrix
P.set(jobclass[0], jobclass[0], node[0], node[0], 0.3)
P.set(jobclass[0], jobclass[0], node[0], node[1], 0.1)  
P.set(jobclass[0], jobclass[0], node[1], node[0], 0.2)
P.set(jobclass[0], jobclass[0], node[1], node[1], 0.0)

# Class 1 to Class 2 routing matrix
P.set(jobclass[0], jobclass[1], node[0], node[0], 0.6)
P.set(jobclass[0], jobclass[1], node[0], node[1], 0.0)
P.set(jobclass[0], jobclass[1], node[1], node[0], 0.8)
P.set(jobclass[0], jobclass[1], node[1], node[1], 0.0)

# Class 2 to Class 2 routing matrix  
P.set(jobclass[1], jobclass[1], node[0], node[0], 0.0)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[0], 0.0)
P.set(jobclass[1], jobclass[1], node[1], node[1], 0.0)

# Class 2 to Class 1 routing matrix
P.set(jobclass[1], jobclass[0], node[0], node[0], 0.0)
P.set(jobclass[1], jobclass[0], node[0], node[1], 0.0)
P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)
P.set(jobclass[1], jobclass[0], node[1], node[1], 0.0)

model.link(P)

In [3]:
# Note: The initFromMarginalAndStarted method is not yet available in Python LINE
# This example demonstrates the 2-class 2-node class-switching model setup

# CTMC is available in the Python wrapper but may not be imported by default
try:
    from line_solver.solvers import CTMC
    solver_ctmc = CTMC(model)
    avg_table_ctmc = solver_ctmc.avg_table()
except (ImportError, Exception) as e:
    print(f"CTMC not available or failed: {e}")

       Station JobClass       QLen       Util   RespT     ResidT       ArvR       Tput
InfiniteServer   Class1 9.8851e-02 9.8851e-02  0.3333 3.3333e-01 2.9655e-01 2.9655e-01
InfiniteServer   Class2 1.6436e-18 1.6436e-18  2.0000 3.1781e-18 8.2178e-19 8.2178e-19
        Queue1   Class1 2.9011e+00 7.4138e-01 19.5658 9.7829e+00 1.4828e-01 1.4828e-01
        Queue1   Class2 1.0000e+00 2.5862e-01  1.9337 1.9337e+00 5.1715e-01 5.1715e-01


In [4]:
solver_jmt = JMT(model, samples=100000, seed=23000)
avg_table_jmt = solver_jmt.avg_table()

       Station JobClass   QLen   Util   RespT  ResidT   ArvR   Tput
InfiniteServer   Class1 0.3515 0.3515  0.3315  0.3315 1.0687 1.0687
InfiniteServer   Class2 1.2924 1.2924  1.9885  1.9885 0.6466 0.6466
        Queue1   Class1 1.4593 0.5393 13.6529 13.6529 0.1077 0.1077
        Queue1   Class2 0.8791 0.3169  1.3689  1.3689 0.6466 0.6466


In [5]:
solver_ssa = SSA(model, samples=100000, seed=23000)
avg_table_ssa = solver_ssa.avg_table()

       Station JobClass   QLen   Util   RespT  ResidT   ArvR   Tput
InfiniteServer   Class1 0.3385 0.3385  0.3330  0.3330 1.0166 1.0166
InfiniteServer   Class2 1.3744 1.3744  1.9869  1.9869 0.6917 0.6917
        Queue1   Class1 1.3601 0.5042 13.5232 13.5232 0.1006 0.1006
        Queue1   Class2 0.9270 0.3432  1.3402  1.3402 0.6917 0.6917


In [6]:
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()

       Station JobClass   QLen   Util   RespT  ResidT   ArvR   Tput
InfiniteServer   Class1 0.3865 0.3865  0.3333  0.3333 1.1595 1.1595
InfiniteServer   Class2 1.5769 1.5769  2.0000  2.0000 0.7884 0.7884
        Queue1   Class1 1.2123 0.5952 10.1832 10.1832 0.1190 0.1190
        Queue1   Class2 0.8244 0.4048  1.0183  1.0183 0.8095 0.8095


In [7]:
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()

       Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
InfiniteServer   Class1 0.2003 0.2003 0.1984  0.1984 1.0096 1.0096
InfiniteServer   Class2 0.5558 0.5558 0.8095  0.8095 0.6865 0.6865
        Queue1   Class1 0.6330 0.1217 6.2697  6.2697 0.1010 0.1010
        Queue1   Class2 0.4304 0.8274 0.6270  0.6270 0.6865 0.6865


In [8]:
# NC solver has limitations on multi-station load-dependent models
try:
    solver_nc = NC(model)
    avg_table_nc = solver_nc.avg_table()
    print(avg_table_nc)
except Exception as e:
    print(f"NC solver limitation: {type(e).__name__} - This model exceeds NC solver capabilities")

       Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
InfiniteServer   Class1 0.9505 0.3143 1.0079  1.0079 0.9430 0.9430
InfiniteServer   Class2 0.6463 1.2825 1.0079  1.0079 0.6412 0.6412
        Queue1   Class1 0.3081 0.4715 3.2672  3.2672 0.0943 0.0943
        Queue1   Class2 2.0951 0.3206 3.2672  3.2672 0.6412 0.6412
          Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  InfiniteServer   Class1  0.9505  0.3143  1.0079  1.0079  0.9430  0.9430
1  InfiniteServer   Class2  0.6463  1.2825  1.0079  1.0079  0.6412  0.6412
2          Queue1   Class1  0.3081  0.4715  3.2672  3.2672  0.0943  0.0943
3          Queue1   Class2  2.0951  0.3206  3.2672  3.2672  0.6412  0.6412
